This exercise is based on the ColabFold notebook. ColabFold predicts structures using AlphaFold2, a groundbreaking protein structure prediction method based on machine learning. It is based on a faster method for multiple sequence alignment than AlphaFold2.

The exercise is intended to help you achieve the following learning objective:
* Predict protein structure based on the sequence of amino acids. Express confidence in the quality of a structure prediction.

When you are done with this exercise, save it under `Chem456-2022F/exercises` on Google Drive. It will be graded as satisfactory or unsatisfactory based on correctly completing the sections after # -->.

# Overview

The western corn rootworm (WCR) is an important agricultural pest that has evolved the ability to tolerate the plant defense toxin DIMBOA. Nick Miller (Biology) and I are interested in identifying WCR enzymes that metabolize DIMBOA. We hypothesize that detoxification is performed by cytochrome P450s, of which WCR has 85. In this exercise, you will build a model of two WCR enzymes.

In Part I, you will use ColabFold to model the structure of one enzyme without cofactors. You will also assess this model and compare it to a model from AlphaFold2, which is based on a slower multiple sequence alignment. In Part II, you will model the position of the heme cofactor based on a homologous crystal structure.

You will find the protein sequences, AlphaFold2 structures, and your assigned proteins in this folder:
https://drive.google.com/drive/folders/1BLshPMhersj9C6WzPyPIdoP-iBKhC_XO?usp=sharing

If you are working on this exercise independently (opposed to being in Chem 456), you may try the sequence of the WCR CYP4AA5:
`MDYYLGYILTVLLTIFRTLKNYMRTVILAWKLPGPTALPILGNVLILKNHTELIYLGNHAPNMYGKFFRVWCSILPAIFITEPRHLKQILSTNKNNEKSLFYRAFHNFIGEGLITNNGYKWKKNRKMIQPYFHMNILEKFVDLFTECSSRFVSKLEGEQTVKITPFINDCVLDILHNGVLGVPFDQDSPYRKGQLQAIERFVRPWLLFDSIFDRTNSANYEKKQKHNLHSYTEEVLNNRRQEIKKGTKGTKTCFLDMFIEIADNNSQFTEEDIINEIVTFMLAGQDSVGATIAFSMYYLAKYPEIQDKVLKEISGIDATQKISIKELNEMKYLEQVVKETLRLTPAVPIITRVLTEDVTLDETVFPNGTIIFISPFMTQRLPHVFPDPNTFNPDRFEESNLENIHPYAFLPFSLGPRNCIGYKFAMIEVKTVLFYVLKKFQLSPAKGKEELSLSYRATIRASGGIWVNLKHRKS`

<img src="https://raw.githubusercontent.com/sokrypton/ColabFold/main/.github/ColabFold_Marv_Logo_Small.png" height="200" align="right" style="height:240px">

# Part I. ColabFold

Easy to use protein structure and complex prediction using [AlphaFold2](https://www.nature.com/articles/s41586-021-03819-2) and [Alphafold2-multimer](https://www.biorxiv.org/content/10.1101/2021.10.04.463034v1). Sequence alignments/templates are generated through [MMseqs2](mmseqs.com) and [HHsearch](https://github.com/soedinglab/hh-suite). For more details, see <a href="#Instructions">bottom</a> of the notebook, checkout the [ColabFold GitHub](https://github.com/sokrypton/ColabFold) and read the manuscript. Old versions: [v1.0](https://colab.research.google.com/github/sokrypton/ColabFold/blob/v1.0-alpha/AlphaFold2.ipynb), [v1.1](https://colab.research.google.com/github/sokrypton/ColabFold/blob/v1.1-premultimer/AlphaFold2.ipynb), [v1.2](https://colab.research.google.com/github/sokrypton/ColabFold/blob/v1.2.0/AlphaFold2.ipynb), [v1.3](https://colab.research.google.com/github/sokrypton/ColabFold/blob/v1.3.0/AlphaFold2.ipynb)

[Mirdita M, Schütze K, Moriwaki Y, Heo L, Ovchinnikov S, Steinegger M. ColabFold: Making protein folding accessible to all.
*Nature Methods*, 2022](https://www.nature.com/articles/s41592-022-01488-1) 

In [ ]:
#@title Input protein sequence(s), then hit `Runtime` -> `Run all`
from google.colab import files
import os.path
import re
import hashlib
import random

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

query_sequence = 'MDYYLGYILTVLLTIFRTLKNYMRTVILAWKLPGPTALPILGNVLILKNHTELIYLGNHAPNMYGKFFRVWCSILPAIFITEPRHLKQILSTNKNNEKSLFYRAFHNFIGEGLITNNGYKWKKNRKMIQPYFHMNILEKFVDLFTECSSRFVSKLEGEQTVKITPFINDCVLDILHNGVLGVPFDQDSPYRKGQLQAIERFVRPWLLFDSIFDRTNSANYEKKQKHNLHSYTEEVLNNRRQEIKKGTKGTKTCFLDMFIEIADNNSQFTEEDIINEIVTFMLAGQDSVGATIAFSMYYLAKYPEIQDKVLKEISGIDATQKISIKELNEMKYLEQVVKETLRLTPAVPIITRVLTEDVTLDETVFPNGTIIFISPFMTQRLPHVFPDPNTFNPDRFEESNLENIHPYAFLPFSLGPRNCIGYKFAMIEVKTVLFYVLKKFQLSPAKGKEELSLSYRATIRASGGIWVNLKHRKS' #@param {type:"string"}
#@markdown  - Use `:` to specify inter-protein chainbreaks for **modeling complexes** (supports homo- and hetro-oligomers). For example **PI...SK:PI...SK** for a homodimer

# remove whitespaces
query_sequence = "".join(query_sequence.split())

jobname = 'CYP4AA5' #@param {type:"string"}
# remove whitespaces
basejobname = "".join(jobname.split())
basejobname = re.sub(r'\W+', '', basejobname)
jobname = add_hash(basejobname, query_sequence)
while os.path.isfile(f"{jobname}.csv"):
  jobname = add_hash(basejobname, ''.join(random.sample(query_sequence,len(query_sequence))))

with open(f"{jobname}.csv", "w") as text_file:
    text_file.write(f"id,sequence\n{jobname},{query_sequence}")

queries_path=f"{jobname}.csv"

# number of models to use
use_amber = False #@param {type:"boolean"}
template_mode = "pdb70" #@param ["none", "pdb70","custom"]
#@markdown - "none" = no template information is used, "pdb70" = detect templates in pdb70, "custom" - upload and search own templates (PDB or mmCIF format, see [notes below](#custom_templates))

if template_mode == "pdb70":
  use_templates = True
  custom_template_path = None
elif template_mode == "custom":
  custom_template_path = f"{jobname}_template"
  os.mkdir(custom_template_path)
  uploaded = files.upload()
  use_templates = True
  for fn in uploaded.keys():
    os.rename(fn, f"{jobname}_template/{fn}")
else:
  custom_template_path = None
  use_templates = False


In [ ]:
#@markdown ### MSA options (custom MSA upload, single sequence, pairing mode)
msa_mode = "MMseqs2 (UniRef+Environmental)" #@param ["MMseqs2 (UniRef+Environmental)", "MMseqs2 (UniRef only)","single_sequence","custom"]
pair_mode = "unpaired+paired" #@param ["unpaired+paired","paired","unpaired"] {type:"string"}
#@markdown - "unpaired+paired" = pair sequences from same species + unpaired MSA, "unpaired" = seperate MSA for each chain, "paired" - only use paired sequences.

# decide which a3m to use
if msa_mode.startswith("MMseqs2"):
  a3m_file = f"{jobname}.a3m"
elif msa_mode == "custom":
  a3m_file = f"{jobname}.custom.a3m"
  if not os.path.isfile(a3m_file):
    custom_msa_dict = files.upload()
    custom_msa = list(custom_msa_dict.keys())[0]
    header = 0
    import fileinput
    for line in fileinput.FileInput(custom_msa,inplace=1):
      if line.startswith(">"):
         header = header + 1
      if not line.rstrip():
        continue
      if line.startswith(">") == False and header == 1:
         query_sequence = line.rstrip()
      print(line, end='')

    os.rename(custom_msa, a3m_file)
    queries_path=a3m_file
    print(f"moving {custom_msa} to {a3m_file}")
else:
  a3m_file = f"{jobname}.single_sequence.a3m"
  with open(a3m_file, "w") as text_file:
    text_file.write(">1\n%s" % query_sequence)

In [ ]:
#@markdown ### Advanced settings
model_type = "auto" #@param ["auto", "AlphaFold2-ptm", "AlphaFold2-multimer-v1", "AlphaFold2-multimer-v2"]
#@markdown - "auto" = protein structure prediction using "AlphaFold2-ptm" and complex prediction "AlphaFold-multimer-v2". For complexes "AlphaFold-multimer-v[1,2]" and "AlphaFold-ptm" can be used.
num_recycles = 3 #@param [1,3,6,12,24,48] {type:"raw"}
save_to_google_drive = True #@param {type:"boolean"}

#@markdown -  if the save_to_google_drive option was selected, the result zip will be uploaded to your Google Drive
dpi = 200 #@param {type:"integer"}
#@markdown - set dpi for image resolution

#@markdown Don't forget to hit `Runtime` -> `Run all` after updating the form.

# We will use google.colab instead of pydrive, which was in the original ColabFold
# if save_to_google_drive:
#   from pydrive.drive import GoogleDrive
#   from pydrive.auth import GoogleAuth
#   from google.colab import auth
#   from oauth2client.client import GoogleCredentials
#   auth.authenticate_user()
#   gauth = GoogleAuth()
#   gauth.credentials = GoogleCredentials.get_application_default()
#   drive = GoogleDrive(gauth)
#   print("You are logged into Google Drive and are good to go!")

In [ ]:
#@title Install dependencies using python
#@markdown If necessary, mount Google Drive. Install py3Dmol for structural visualization and prody for structural alignment.

from google.colab import drive
if not os.path.isdir('/content/drive'):
  drive.mount('/content/drive')

try:
  import py3Dmol
except:
  !pip install py3Dmol
  import py3Dmol
  
try:
  import prody
except:
  !pip install -U prody
  import prody

In [ ]:
#@title Install dependencies using bash
#@markdown If you have already completed Part I, you can skip ahead to Part II without running this cell.
%%bash -s $use_amber $use_templates

set -e

USE_AMBER=$1
USE_TEMPLATES=$2

if [ ! -f COLABFOLD_READY ]; then
  # install dependencies
  # We have to use "--no-warn-conflicts" because colab already has a lot preinstalled with requirements different to ours
  pip install -q --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold"
  # high risk high gain
  pip install -q "jax[cuda11_cudnn805]>=0.3.8,<0.4" -f https://storage.googleapis.com/jax-releases/jax_releases.html
  touch COLABFOLD_READY
fi

# setup conda
if [ ${USE_AMBER} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f CONDA_READY ]; then
    wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Miniconda3-latest-Linux-x86_64.sh
    touch CONDA_READY
  fi
fi
# setup template search
if [ ${USE_TEMPLATES} == "True" ] && [ ! -f HH_READY ]; then
  conda install -y -q -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 python=3.7 2>&1 1>/dev/null
  touch HH_READY
fi
# setup openmm for amber refinement
if [ ${USE_AMBER} == "True" ] && [ ! -f AMBER_READY ]; then
  conda install -y -q -c conda-forge openmm=7.5.1 python=3.7 pdbfixer 2>&1 1>/dev/null
  touch AMBER_READY
fi

In [ ]:
#@title Run Prediction

import sys

from colabfold.download import download_alphafold_params, default_data_dir
from colabfold.utils import setup_logging
from colabfold.batch import get_queries, run, set_model_type
K80_chk = !nvidia-smi | grep "Tesla K80" | wc -l
if "1" in K80_chk:
  print("WARNING: found GPU Tesla K80: limited to total length < 1000")
  if "TF_FORCE_UNIFIED_MEMORY" in os.environ:
    del os.environ["TF_FORCE_UNIFIED_MEMORY"]
  if "XLA_PYTHON_CLIENT_MEM_FRACTION" in os.environ:
    del os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]

from colabfold.colabfold import plot_protein
from pathlib import Path
import matplotlib.pyplot as plt


# For some reason we need that to get pdbfixer to import
if use_amber and '/usr/local/lib/python3.7/site-packages/' not in sys.path:
    sys.path.insert(0, '/usr/local/lib/python3.7/site-packages/')

def prediction_callback(unrelaxed_protein, length, prediction_result, input_features, type):
  fig = plot_protein(unrelaxed_protein, Ls=length, dpi=150)
  plt.show()
  plt.close()

result_dir="."
if 'logging_setup' not in globals():
    setup_logging(Path(".").joinpath("log.txt"))
    logging_setup = True

queries, is_complex = get_queries(queries_path)
model_type = set_model_type(is_complex, model_type)
download_alphafold_params(model_type, Path("."))
run(
    queries=queries,
    result_dir=result_dir,
    use_templates=use_templates,
    custom_template_path=custom_template_path,
    use_amber=use_amber,
    msa_mode=msa_mode,    
    model_type=model_type,
    num_models=5,
    num_recycles=num_recycles,
    model_order=[1, 2, 3, 4, 5],
    is_complex=is_complex,
    data_dir=Path("."),
    keep_existing_results=False,
    recompile_padding=1.0,
    rank_by="auto",
    pair_mode=pair_mode,
    stop_at_score=float(100),
    prediction_callback=prediction_callback,
    dpi=dpi
)

In [ ]:
#@title Display 3D structure {run: "auto"}
import py3Dmol
import glob
import matplotlib.pyplot as plt
from colabfold.colabfold import plot_plddt_legend
rank_num = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
color = "lDDT" #@param ["chain", "lDDT", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}

jobname_prefix = ".custom" if msa_mode == "custom" else ""
if use_amber:
  pdb_filename = f"{jobname}{jobname_prefix}_relaxed_rank_{rank_num}_model_*.pdb"
else:
  pdb_filename = f"{jobname}{jobname_prefix}_unrelaxed_rank_{rank_num}_model_*.pdb"

pdb_file = glob.glob(pdb_filename)

def show_pdb(rank_num=1, show_sidechains=False, show_mainchains=False, color="lDDT"):
  model_name = f"rank_{rank_num}"
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
  view.addModel(open(pdb_file[0],'r').read(),'pdb')

  if color == "lDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':50,'max':90}}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  elif color == "chain":
    chains = len(queries[0][1]) + 1 if is_complex else 1
    for n,chain,color in zip(range(chains),list("ABCDEFGH"),
                     ["lime","cyan","magenta","yellow","salmon","white","blue","orange"]):
      view.setStyle({'chain':chain},{'cartoon': {'color':color}})
  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})  
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})

  view.zoomTo()
  return view


show_pdb(rank_num,show_sidechains, show_mainchains, color).show()
if color == "lDDT":
  plot_plddt_legend().show() 

## # --> Take a screen shot 

that shows the py3Dmol view and level of confidence in your models. Save it in your `03-colab_fold` folder as `CYPID.png`, where `CYPID` is the identifier of your Cytochrome P450.

## --> Enter a short answer in this text box

Summarize the confidence in your models. What regions in the protein is confidence lower?

In [ ]:
#@title Plots {run: "auto"}
from IPython.display import display, HTML
import base64
from html import escape

# see: https://stackoverflow.com/a/53688522
def image_to_data_url(filename):
  ext = filename.split('.')[-1]
  prefix = f'data:image/{ext};base64,'
  with open(filename, 'rb') as f:
    img = f.read()
  return prefix + base64.b64encode(img).decode('utf-8')

pae = image_to_data_url(f"{jobname}{jobname_prefix}_PAE.png")
cov = image_to_data_url(f"{jobname}{jobname_prefix}_coverage.png")
plddt = image_to_data_url(f"{jobname}{jobname_prefix}_plddt.png")
display(HTML(f"""
<style>
  img {{
    float:left;
  }}
  .full {{
    max-width:100%;
  }}
  .half {{
    max-width:50%;
  }}
  @media (max-width:640px) {{
    .half {{
      max-width:100%;
    }}
  }}
</style>
<div style="max-width:90%; padding:2em;">
  <h1>Plots for {escape(jobname)}</h1>
  <img src="{pae}" class="full" />
  <img src="{cov}" class="half" />
  <img src="{plddt}" class="half" />
</div>
"""))


For the Sequence coverage plot, the ColabFold documentation explains that

> Number of sequences per position - We want to see at least 30 sequences per position, for best performance, ideally 100 sequences.
Predicted lDDT per position - model confidence (out of 100) at each position. The higher the better.

## --> Enter a short answer in this text box

For your CYPs, is their adequate sequence coverage for all positions? If not, in what positions is there low coverage?

In [ ]:
#@title Package and download results
#@markdown If you are having issues downloading the result archive, try disabling your adblocker and run this cell again. If that fails click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\" (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).

if msa_mode == "custom":
  print("Don't forget to cite your custom MSA generation method.")

!zip -FSr $jobname".result.zip" config.json $jobname*".json" $jobname*".a3m" $jobname*"relaxed_rank_"*".pdb" "cite.bibtex" $jobname*".png"
# files.download(f"{jobname}.result.zip")

if save_to_google_drive == True:
  from google.colab import drive
  if not os.path.isdir('/content/drive'):
    drive.mount('/content/drive')

  import os
  class_dir = os.path.join('/content','drive','MyDrive','Chem456-2022F')
  if not os.path.isdir(class_dir):
    raise ValueError('Class directory not set up')
  else:
    os.chdir(class_dir)

  labs_dir = os.path.join(class_dir,'labs')
  if not os.path.isdir(labs_dir):
    raise ValueError('Labs directory not set up')
  else:
    os.chdir(labs_dir)

  os.chdir(labs_dir)
  colab_fold_dir = os.path.join(labs_dir,'03-colab_fold')
  if not os.path.isdir(colab_fold_dir):
    os.mkdir('03-colab_fold')
 
  import shutil
  shutil.copy(os.path.join('/content',    f"{jobname}.result.zip"),
              os.path.join(colab_fold_dir,f"{jobname}.result.zip"))
  print(f'Saved {jobname}.result.zip to {colab_fold_dir}')
  os.chdir('/content')

  # uploaded = drive.CreateFile({'title': f"{jobname}.result.zip"})
  # uploaded.SetContentFile(f"{jobname}.result.zip")
  # uploaded.Upload()
  # print(f"Uploaded {jobname}.result.zip to Google Drive with ID {uploaded.get('id')}")

# Instructions <a name="Instructions"></a>
**Quick start**
1. Paste your protein sequence(s) in the input field.
2. Press "Runtime" -> "Run all".
3. The pipeline consists of 5 steps. The currently running step is indicated by a circle with a stop sign next to it.

**Result zip file contents**

1. PDB formatted structures sorted by avg. pLDDT and complexes are sorted by pTMscore. (unrelaxed and relaxed if `use_amber` is enabled).
2. Plots of the model quality.
3. Plots of the MSA coverage.
4. Parameter log file.
5. A3M formatted input MSA.
6. A `predicted_aligned_error_v1.json` using [AlphaFold-DB's format](https://alphafold.ebi.ac.uk/faq#faq-7) and a `scores.json` for each model which contains an array (list of lists) for PAE, a list with the average pLDDT and the pTMscore.
7. BibTeX file with citations for all used tools and databases.

At the end of the job a download modal box will pop up with a `jobname.result.zip` file. Additionally, if the `save_to_google_drive` option was selected, the `jobname.result.zip` will be uploaded to your Google Drive.

**MSA generation for complexes**

For the complex prediction we use unpaired and paired MSAs. Unpaired MSA is generated the same way as for the protein structures prediction by searching the UniRef100 and environmental sequences three iterations each.

The paired MSA is generated by searching the UniRef100 database and pairing the best hits sharing the same NCBI taxonomic identifier (=species or sub-species). We only pair sequences if all of the query sequences are present for the respective taxonomic identifier.

**Using a custom MSA as input**

To predict the structure with a custom MSA (A3M formatted): (1) Change the `msa_mode`: to "custom", (2) Wait for an upload box to appear at the end of the "MSA options ..." box. Upload your A3M. The first fasta entry of the A3M must be the query sequence without gaps. 

It is also possilbe to proide custom MSAs for complex predictions. Read more about the format [here](https://github.com/sokrypton/ColabFold/issues/76).

As an alternative for MSA generation the [HHblits Toolkit server](https://toolkit.tuebingen.mpg.de/tools/hhblits) can be used. After submitting your query, click "Query Template MSA" -> "Download Full A3M". Download the A3M file and upload it in this notebook.

**Using custom templates** <a name="custom_templates"></a>

To predict the structure with a custom template (PDB or mmCIF formatted): (1) change the `template_mode` to "custom" in the execute cell and (2) wait for an upload box to appear at the end of the "Input Protein" box. Select and upload your templates (multiple choices are possible).

* Templates must follow the four letter PDB naming.

* Templates in mmCIF format must contain `_entity_poly_seq`. An error is thrown if this field is not present. The field `_pdbx_audit_revision_history.revision_date` is automatically generated if it is not present.

* Templates in PDB format are automatically converted to the mmCIF format. `_entity_poly_seq` and `_pdbx_audit_revision_history.revision_date` are automatically generated.

If you encounter problems, please report them to this [issue](https://github.com/sokrypton/ColabFold/issues/177).

**Comparison to the full AlphaFold2 and Alphafold2 colab**

This notebook replaces the homology detection and MSA pairing of AlphaFold2 with MMseqs2. For a comparison against the [AlphaFold2 Colab](https://colab.research.google.com/github/deepmind/alphafold/blob/main/notebooks/AlphaFold.ipynb) and the full [AlphaFold2](https://github.com/deepmind/alphafold) system read our [preprint](https://www.biorxiv.org/content/10.1101/2021.08.15.456425v1). 

**Troubleshooting**
* Check that the runtime type is set to GPU at "Runtime" -> "Change runtime type".
* Try to restart the session "Runtime" -> "Factory reset runtime".
* Check your input sequence.

**Known issues**
* Google Colab assigns different types of GPUs with varying amount of memory. Some might not have enough memory to predict the structure for a long sequence.
* Your browser can block the pop-up for downloading the result file. You can choose the `save_to_google_drive` option to upload to Google Drive instead or manually download the result file: Click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\" (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).

**Limitations**
* Computing resources: Our MMseqs2 API can handle ~20-50k requests per day.
* MSAs: MMseqs2 is very precise and sensitive but might find less hits compared to HHblits/HMMer searched against BFD or MGnify.
* We recommend to additionally use the full [AlphaFold2 pipeline](https://github.com/deepmind/alphafold).

**Description of the plots**
*   **Number of sequences per position** - We want to see at least 30 sequences per position, for best performance, ideally 100 sequences.
*   **Predicted lDDT per position** - model confidence (out of 100) at each position. The higher the better.
*   **Predicted Alignment Error** - For homooligomers, this could be a useful metric to assess how confident the model is about the interface. The lower the better.

**Bugs**
- If you encounter any bugs, please report the issue to https://github.com/sokrypton/ColabFold/issues

**License**

The source code of ColabFold is licensed under [MIT](https://raw.githubusercontent.com/sokrypton/ColabFold/main/LICENSE). Additionally, this notebook uses the AlphaFold2 source code and its parameters licensed under [Apache 2.0](https://raw.githubusercontent.com/deepmind/alphafold/main/LICENSE) and [CC BY 4.0](https://creativecommons.org/licenses/by-sa/4.0/) respectively. Read more about the AlphaFold license [here](https://github.com/deepmind/alphafold).

**Acknowledgments**
- We thank the AlphaFold team for developing an excellent model and open sourcing the software. 

- [KOBIC](https://kobic.re.kr) and [Söding Lab](https://www.mpinat.mpg.de/soeding) for providing the computational resources for the MMseqs2 MSA server.

- Richard Evans for helping to benchmark the ColabFold's Alphafold-multimer support.

- [David Koes](https://github.com/dkoes) for his awesome [py3Dmol](https://3dmol.csb.pitt.edu/) plugin, without whom these notebooks would be quite boring!

- Do-Yoon Kim for creating the ColabFold logo.

- A colab by Sergey Ovchinnikov ([@sokrypton](https://twitter.com/sokrypton)), Milot Mirdita ([@milot_mirdita](https://twitter.com/milot_mirdita)) and Martin Steinegger ([@thesteinegger](https://twitter.com/thesteinegger)).


# Part II. Heme placement

If you run Part II separately from Pa
rt I, you will need to upload your results from Part I to the Colab instance and decompress it.

In [ ]:
# !unzip {jobname}.result.zip

## Compare models from ColabFold and AlphaFold

Let's compare the homology model from ColabFold to the model from AlphaFold. The difference between the two methods is the MSA. You should upload the AlphaFold model to your Google Drive `MyDrive/Chem456-2022F/exercises/03-colab_fold` or add a link to it there.

In [ ]:
# Points to the PDB file from AlphaFold
colab_fold_dir = os.path.join('/content','drive','MyDrive','Chem456-2022F', \
                                'exercise','03-colab_fold')

AlphaFold_PDBFN = os.path.join(colab_fold_dir, f'{basejobname}-m1.pdb')

# Point to the best model from ColabFold
for FN in os.listdir('/content'):
  if FN.find('unrelaxed_rank_1')>-1 and FN.endswith('pdb'):
    ColabFold_PDBFN = os.path.join('/content',FN)

print(f'Best ColabFold model is {ColabFold_PDBFN}')

Aligned_ColabFold_PDBFN = os.path.join(colab_fold_dir, \
                                       f'{basejobname}_ColabFold.pdb')

In [ ]:
# Align the ColabFold to the AlphaFold model
print(f'Loading AlphaFold model from {AlphaFold_PDBFN}')
AlphaFold_PDB = prody.parsePDB(AlphaFold_PDBFN)
print(f'Loading ColabFold model from {ColabFold_PDBFN}')
ColabFold_PDB = prody.parsePDB(ColabFold_PDBFN)

AlphaFold_PDB_prot = AlphaFold_PDB.select('protein')
ColabFold_PDB_prot = ColabFold_PDB.select('protein')

map_AC, map_CA, seqid, overlap = \
  prody.matchChains(AlphaFold_PDB_prot, ColabFold_PDB_prot)[0]
transformation = prody.calcTransformation(map_CA, map_AC)

prody.applyTransformation(transformation, ColabFold_PDB)
prody.writePDB(Aligned_ColabFold_PDBFN, ColabFold_PDB)

In [ ]:
view = py3Dmol.view()
view.addModel(open(AlphaFold_PDBFN, 'r').read(),'pdb')
view.addModel(open(Aligned_ColabFold_PDBFN, 'r').read(),'pdb')
view.setBackgroundColor('white')
view.setStyle({'model':0}, {'cartoon': {'color':'purple'}})
view.setStyle({'model':1}, {'cartoon': {'color':'yellow'}})
# view.addStyle({'model':0}, {'stick': {'colorscheme':'purpleCarbon'}})
# view.addStyle({'model':1}, {'stick': {'colorscheme':'yellowCarbon'}})
view.zoomTo()
view.show()

## # --> Take a screen shot 

that shows the py3Dmol view of the aligned models. Save it in your `03-colab_fold` folder as `CYPID_model_comparison.png`, where `CYPID` is the identifier of your Cytochrome P450.

## # --> Enter a short answer in this text box

For each of your CYPs, compare and contrast the models from ColabFold and AlphaFold.

## Align the closest template to the AlphaFold model

In [ ]:
# Template file name
Aligned_template_PDBFN = os.path.join(colab_fold_dir, f'{basejobname}_template.pdb')

Aligned_heme_PDBFN = os.path.join(colab_fold_dir, f'{basejobname}_heme.pdb')

In [ ]:
# Identify the closest template from the ColabFold data files
os.chdir('/content')
import json
with open(f"{jobname}_template_domain_names.json",'r') as F:
  templates = json.loads(F.read())
(template_PDBID, template_chain) = templates['A'][0].split('_')

In [ ]:
# Align the template to the AlphaFold model
AlphaFold_PDB = prody.parsePDB(AlphaFold_PDBFN)
template_PDB = prody.parsePDB(template_PDBID)

AlphaFold_PDB_prot = AlphaFold_PDB.select('protein')
template_PDB_prot = template_PDB.select('protein')

map_AT, map_TA, seqid, overlap = \
  prody.matchChains(AlphaFold_PDB_prot, template_PDB_prot, seqid=10, overlap=10)[0]
transformation = prody.calcTransformation(map_TA, map_AT)

prody.applyTransformation(transformation, template_PDB)
prody.writePDB(Aligned_template_PDBFN, template_PDB)

In [ ]:
print(f'The closest template is {template_PDBID}.')
print('These are the unique residues in the template:')
print(' '.join(set(list(template_PDB.getResnames()))))

## # --> Enter a short answer in this text box

For each of your CYPs, which residues are not standard amino acids? Three-letter codes are widely available [online](https://www.fao.org/3/Y2775E/y2775e0e.htm). You may also consult the [PDB](https://www.rcsb.org/).

In [ ]:
# --> Enter the residue name of the heme. It is probably 'HEM'
heme_resname = 'HEM'

The code below should show the AlphaFold model, the closest template, and the heme. If not, make sure that the style has the correct residue name for your template.

In [ ]:
template_PDB_heme = template_PDB.select(f'resname {heme_resname}')
prody.writePDB(Aligned_heme_PDBFN, template_PDB_heme)

In [ ]:
view = py3Dmol.view()
view.addModel(open(AlphaFold_PDBFN, 'r').read(),'pdb')
view.addModel(open(Aligned_template_PDBFN, 'r').read(),'pdb')
view.setBackgroundColor('white')
view.setStyle({'model':0}, {'cartoon': {'color':'purple'}})
view.setStyle({'model':1}, {'cartoon': {'color':'yellow'}})
view.addStyle({'resn':'HEM'}, {'sphere': {'colorscheme':'yellowCarbon'}})
# view.addStyle({'model':0}, {'stick': {'colorscheme':'purpleCarbon'}})
# view.addStyle({'model':1}, {'stick': {'colorscheme':'yellowCarbon'}})
view.zoomTo()
view.show()

## # --> Take a screen shot 

that shows the py3Dmol view of the aligned models and shows the heme. Save it in your `03-colab_fold` folder as `CYPID_with_heme.png`, where `CYPID` is the identifier of your Cytochrome P450.

In [ ]:
# This code should join the AlphaFold model with the heme from the template
AlphaFoldHeme_PDBFN = os.path.join(colab_fold_dir, f'{basejobname}_with_heme.pdb')
!cat {AlphaFold_PDBFN} {Aligned_heme_PDBFN} > {AlphaFoldHeme_PDBFN}